In [1]:
import scrapy
from selenium import webdriver
from fake_useragent import UserAgent

In [2]:
!scrapy startproject naverflight

Error: scrapy.cfg already exists in /home/ubuntu/python3/notebook/assgn/naverflight


#### 1. items.py 작성

In [3]:
!cat naverflight/naverflight/items.py

import scrapy


class CrawlerItem(scrapy.Item):
    now = scrapy.Field() # 크롤링 시각
    flight = scrapy.Field() # 항공편
    departure = scrapy.Field() # 출발지
    tod = scrapy.Field() # 출발시각
    t_time = scrapy.Field() # 총 비행시간
    arrival = scrapy.Field() # 도착지
    toa = scrapy.Field() # 도착시각
    transfer = scrapy.Field() # 경유 회수
    price = scrapy.Field() # 항공권 가격


In [4]:
%%writefile naverflight/naverflight/items.py
import scrapy


class CrawlerItem(scrapy.Item):
    now = scrapy.Field() # 크롤링 시각
    flight = scrapy.Field() # 항공편
    departure = scrapy.Field() # 출발지
    tod = scrapy.Field() # 출발시각
    t_time = scrapy.Field() # 총 비행시간
    arrival = scrapy.Field() # 도착지
    toa = scrapy.Field() # 도착시각
    transfer = scrapy.Field() # 경유 회수
    price = scrapy.Field() # 항공권 가격

Overwriting naverflight/naverflight/items.py


#### 2. spider.py 작성

In [15]:
%%writefile naverflight/naverflight/spiders/spider.py
import scrapy
from naverflight.items import CrawlerItem
from selenium import webdriver
from fake_useragent import UserAgent
import time

class Spider(scrapy.Spider): # scrapy.Spider 상속받고
    name = "naverflight"
    allow_domain = ["naver.com"] # 이 도메인으로만 크롤링하 겠다. 여러개 작성가능
    start_urls = ["https://flight.naver.com/flights/v2/results?trip=RT&scity1=ICN&ecity1=KEF&scity2=KEF&ecity2=ICN&adult=1&child=0&infant=0&sdate1=2020.02.19.&sdate2=2020.02.25.&fareType=Y"] # 여러개 가능
    
    def parse(self, response): #response를 받아서 이 함수를 실행하는 것. 중간과정(request)은 프레임워크안에 있어서 생략
        item = CrawlerItem()
        options = webdriver.ChromeOptions()
        options.add_argument("headless")
        options.add_argument("user-agent={}".format(UserAgent().chrome))
        driver = webdriver.Chrome(options=options)
        url = "https://flight.naver.com/flights/v2/results?trip=RT&scity1=ICN&ecity1=KEF&scity2=KEF&ecity2=ICN&adult=1&child=0&infant=0&sdate1=2020.02.19.&sdate2=2020.02.25.&fareType=Y"
        driver.get(url)
        time.sleep(10)
        item["now"] = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(time.time()))
        flight = driver.find_elements_by_css_selector(".h_result > .ng-binding")
        item["flight"] = flight[0].text
        departure = driver.find_elements_by_css_selector(".box_route > .route_info_box > .route_info > .txt_code")
        item["departure"] = departure[0].text
        tod = driver.find_elements_by_css_selector(".box_route > .route_info_box > .route_info > .txt_time")
        item["tod"] = tod[0].text
        t_time = driver.find_elements_by_css_selector(".box_route > .route_info_box > .route_info.route_info_time > .txt_time")
        item["t_time"] = t_time[0].text
        arrival = driver.find_elements_by_css_selector(".box_route > .route_info_box > .route_info:nth-child(2) > .txt_code")
        item["arrival"] = arrival[0].text
        toa = driver.find_elements_by_css_selector(".box_route > .route_info_box > .route_info:nth-child(2) > .txt_time")
        item["toa"] = toa[0].text
        transfer = driver.find_elements_by_css_selector(".box_info > .txt_way > .btn_pay_terms")
        item["transfer"] = transfer[0].text
        price = driver.find_elements_by_css_selector(".txt_bill > .txt_total > .ng-binding")
        item["price"] = price[0].text
        driver.quit()
        yield item

Overwriting naverflight/naverflight/spiders/spider.py


#### 3. Scrapy 실행

In [16]:
%%writefile run.sh
#!/bin/bash
cd ~/python3/notebook/assgn/naverflight
scrapy crawl naverflight -o naverflight.csv

Overwriting run.sh


In [17]:
!chmod +x run.sh

In [18]:
!head -n 25 naverflight/naverflight/settings.py | tail -n 5

# Obey robots.txt rules
ROBOTSTXT_OBEY = False

# Configure maximum concurrent requests performed by Scrapy (default: 16)
#CONCURRENT_REQUESTS = 32


In [19]:
!sed -i 's/ROBOTSTXT_OBEY = True/ROBOTSTXT_OBEY = False/' naverflight/naverflight/settings.py

In [27]:
# !./run.sh

2019-12-10 13:35:45 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: naverflight)
2019-12-10 13:35:45 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Oct 24 2019, 05:23:50) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1054-aws-x86_64-with-debian-buster-sid
2019-12-10 13:35:45 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naverflight', 'FEED_FORMAT': 'csv', 'FEED_URI': 'naverflight.csv', 'NEWSPIDER_MODULE': 'naverflight.spiders', 'SPIDER_MODULES': ['naverflight.spiders']}
2019-12-10 13:35:46 [scrapy.extensions.telnet] INFO: Telnet Password: 7c3a6c36cc0cbcda
2019-12-10 13:35:46 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.log

2019-12-10 13:36:00 [urllib3.connectionpool] DEBUG: http://127.0.0.1:47277 "POST /session/a331b2f47fc49813eb42649ef1fef36d/elements HTTP/1.1" 200 1591
2019-12-10 13:36:00 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2019-12-10 13:36:00 [selenium.webdriver.remote.remote_connection] DEBUG: GET http://127.0.0.1:47277/session/a331b2f47fc49813eb42649ef1fef36d/element/8e3f3c49-b9ad-495e-89b8-4b19d0e7243d/text {"id": "8e3f3c49-b9ad-495e-89b8-4b19d0e7243d"}
2019-12-10 13:36:00 [urllib3.connectionpool] DEBUG: http://127.0.0.1:47277 "GET /session/a331b2f47fc49813eb42649ef1fef36d/element/8e3f3c49-b9ad-495e-89b8-4b19d0e7243d/text HTTP/1.1" 200 15
2019-12-10 13:36:00 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2019-12-10 13:36:00 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:47277/session/a331b2f47fc49813eb42649ef1fef36d/elements {"using": "css selector", "value": ".box_route > .route_info_box > .route_info:nth-child(2)

In [28]:
# import pandas as pd

In [30]:
# df = pd.read_csv("naverflight/naverflight.csv")
# df.tail()

,arrival,departure,flight,now,price,t_time,toa,tod,transfer
2,KEF,ICN,KLM네덜란드항공,2019-12-10 12:32:09,"905,200",27시간 10분,19:05,00:55,경유 1회
3,arrival,departure,flight,now,price,t_time,toa,tod,transfer
4,KEF,ICN,KLM네덜란드항공,2019-12-10 12:33:11,"905,200",27시간 10분,19:05,00:55,경유 1회
5,arrival,departure,flight,now,price,t_time,toa,tod,transfer
6,KEF,ICN,KLM네덜란드항공,2019-12-10 13:36:00,"905,200",27시간 10분,19:05,00:55,경유 1회


#### 4 .Pipelines 설정
- item을 출력하기전에 실행되는 코드를 정의

In [20]:
%%writefile naverflight/naverflight/mongodb.py
import pymongo


client = pymongo.MongoClient('mongodb://54.180.51.121:27017')
db = client.naverflight
collection = db.flight

Overwriting naverflight/naverflight/mongodb.py


In [21]:
%%writefile naverflight/naverflight/pipelines.py
from .mongodb import collection


class naverflightPipeline(object):
    
    def process_item(self, item, spider):
        
        data = {"now":item["now"], "flight":item["flight"], "departure":item["departure"], "tod":item["tod"],  "t_time":item["t_time"],  "arrival":item["arrival"],  "toa":item["toa"],  "transfer":item["transfer"],  "price":item["price"]}
        collection.insert(data)
        
        return item

Overwriting naverflight/naverflight/pipelines.py


In [22]:
!echo "ITEM_PIPELINES = {" >> naverflight/naverflight/settings.py
!echo "    'naverflight.pipelines.naverflightPipeline' : 300," >> naverflight/naverflight/settings.py
!echo "}" >> naverflight/naverflight/settings.py

In [23]:
!tail -n 3 naverflight/naverflight/settings.py

ITEM_PIPELINES = {
    'naverflight.pipelines.naverflightPipeline' : 300,
}


In [25]:
!./run.sh

2019-12-11 00:25:52 [scrapy.utils.log] INFO: Scrapy 1.8.0 started (bot: naverflight)
2019-12-11 00:25:52 [scrapy.utils.log] INFO: Versions: lxml 4.4.1.0, libxml2 2.9.9, cssselect 1.1.0, parsel 1.5.2, w3lib 1.21.0, Twisted 19.10.0, Python 3.6.9 (default, Oct 24 2019, 05:23:50) - [GCC 7.4.0], pyOpenSSL 19.1.0 (OpenSSL 1.1.1d  10 Sep 2019), cryptography 2.8, Platform Linux-4.15.0-1054-aws-x86_64-with-debian-buster-sid
2019-12-11 00:25:52 [scrapy.crawler] INFO: Overridden settings: {'BOT_NAME': 'naverflight', 'FEED_FORMAT': 'csv', 'FEED_URI': 'naverflight.csv', 'NEWSPIDER_MODULE': 'naverflight.spiders', 'SPIDER_MODULES': ['naverflight.spiders']}
2019-12-11 00:25:52 [scrapy.extensions.telnet] INFO: Telnet Password: d2320365694ca246
2019-12-11 00:25:52 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.log

2019-12-11 00:26:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:41999 "GET /session/c38dad77c6ba414b4ea2300eb0ad87ff/element/5cf58514-8e77-44df-b543-90f478922739/text HTTP/1.1" 200 15
2019-12-11 00:26:06 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2019-12-11 00:26:06 [selenium.webdriver.remote.remote_connection] DEBUG: POST http://127.0.0.1:41999/session/c38dad77c6ba414b4ea2300eb0ad87ff/elements {"using": "css selector", "value": ".box_route > .route_info_box > .route_info:nth-child(2) > .txt_time"}
2019-12-11 00:26:06 [urllib3.connectionpool] DEBUG: http://127.0.0.1:41999 "POST /session/c38dad77c6ba414b4ea2300eb0ad87ff/elements HTTP/1.1" 200 1591
2019-12-11 00:26:06 [selenium.webdriver.remote.remote_connection] DEBUG: Finished Request
2019-12-11 00:26:06 [selenium.webdriver.remote.remote_connection] DEBUG: GET http://127.0.0.1:41999/session/c38dad77c6ba414b4ea2300eb0ad87ff/element/47c9ca18-9d80-4b96-bf47-985bc8969e4a/text {"id": "47c9ca18-9d80-4b96-bf47-